# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [2]:
### Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame
cities=pd.read_csv('Output/cities.csv')
cities.head() 

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Paamiut,61.9940,-49.6678,21.74,80,0,7.23,GL,1612620442
1,1,Kalmar,56.6616,16.3616,24.01,93,75,10.36,SE,1612620444
2,2,Rikitea,-23.1203,-134.9692,77.34,74,100,13.62,PF,1612620446
3,3,Cidreira,-30.1811,-50.2056,72.91,67,77,12.06,BR,1612620448
4,4,Avarua,-21.2078,-159.7750,77.00,94,75,5.75,CK,1612620449


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:

# Configure gmaps with API key.
gmaps.configure(api_key=g_key)


# Convert Humidity to float and store
humidity = cities["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = cities[["Lat", "Lng"]].astype(float)

# # Create a humidity Heatmap layer
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity= max(cities["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [66]:
plt.savefig("Output/heatmap.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
# Narrow down the DataFrame to find the ideal weather condition.


# Filter temperature between 80-90 F.
ideal_weather = cities.loc[(cities["Max Temp"] < 90) & (cities["Max Temp"] > 80)]

# Filter Wind speed less than 8 mph.
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] < 10]

# Filter cloudiness to Zero cloudiness.
ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,66,Kruisfontein,-34.0033,24.7314,80.01,69,0,5.99,ZA,1612620559
123,123,Bahía Blanca,-38.7196,-62.2724,80.24,46,0,8.75,AR,1612620609
161,161,San Juan,-31.5375,-68.5364,85.03,31,0,3.49,AR,1612620643
183,183,Paranaíba,-19.6772,-51.1908,80.56,59,0,6.71,BR,1612620760
198,198,Abu Dhabi,24.4667,54.3667,84.20,53,0,8.05,AE,1612620786
288,288,Poum,-20.2333,164.0167,80.08,68,0,4.34,NC,1612620947
357,357,Yara,20.2753,-76.9533,81.28,60,0,0.92,CU,1612621065
417,417,Yulara,-25.2406,130.9889,80.60,28,0,8.05,AU,1612621037


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
# Store previous dataframe as hotel_df
hotel_df=ideal_weather

# Create a list to hold hotel result
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    city_location = f"{lat},{lng}"

    params = {
        "location": city_location, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel,Hotel Name
66,66,Kruisfontein,-34.0033,24.7314,80.01,69,0,5.99,ZA,1612620559,Oyster Bay House Rental,Oyster Bay House Rental
123,123,Bahía Blanca,-38.7196,-62.2724,80.24,46,0,8.75,AR,1612620609,Hotel Argos,Hotel Argos
161,161,San Juan,-31.5375,-68.5364,85.03,31,0,3.49,AR,1612620643,Hotel Provincial,Hotel Provincial
183,183,Paranaíba,-19.6772,-51.1908,80.56,59,0,6.71,BR,1612620760,Villa Rica Park Hotel GMR,Villa Rica Park Hotel GMR
198,198,Abu Dhabi,24.4667,54.3667,84.20,53,0,8.05,AE,1612620786,Beach Rotana,Beach Rotana
288,288,Poum,-20.2333,164.0167,80.08,68,0,4.34,NC,1612620947,Gîte kajeon,Gîte kajeon
357,357,Yara,20.2753,-76.9533,81.28,60,0,0.92,CU,1612621065,Casa de Margarita Puebla Lorente,Casa de Margarita Puebla Lorente
417,417,Yulara,-25.2406,130.9889,80.60,28,0,8.05,AU,1612621037,Desert Gardens Hotel - Ayers Rock Resort,Desert Gardens Hotel - Ayers Rock Resort


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [62]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=hotel_info
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [65]:
plt.savefig("Output/hotel_info.png")

<Figure size 432x288 with 0 Axes>